In [5]:
#pip install geocoder

SyntaxError: invalid syntax (<ipython-input-5-3aca62285456>, line 2)

In [2]:
import geocoder

lat_lng_coords = None

while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    
latitude = lat_lng_coords[0]
longitutude = lat_lng_coords[1]

NameError: name 'postal_code' is not defined

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [9]:
can_postal_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(can_postal_url).text

can_html = BeautifulSoup(source, 'html.parser')
can_html = can_html.find('table')

In [12]:
col_names = ['Postalcode', 'Borough', 'Neighborhood']
can_df = pd.DataFrame(columns = col_names)

for tr_cell in can_html.find_all('tr'):
    row_data = []
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        can_df.loc[len(can_df)] = row_data
        
can_df.head()

Postalcode           Borough               Neighborhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

In [29]:
df = can_df

df = df.dropna()
df = df[df['Borough'] != 'Not assigned']

not_assigned_ids = df['Neighborhood'] =='Not assigned'
df['Neighborhood'][not_assigned_ids] = df['Borough'][not_assigned_ids]

df['Neighborhood'] = df['Neighborhood'].str.replace(' /', ',')
df.head(100)

Postalcode           Borough  \
2          M3A        North York   
3          M4A        North York   
4          M5A  Downtown Toronto   
5          M6A        North York   
6          M7A  Downtown Toronto   
..         ...               ...   
153        M1X       Scarborough   
156        M4X  Downtown Toronto   
157        M5X  Downtown Toronto   
160        M8X         Etobicoke   
165        M4Y  Downtown Toronto   

                                      Neighborhood  
2                                        Parkwoods  
3                                 Victoria Village  
4                        Regent Park, Harbourfront  
5                 Lawrence Manor, Lawrence Heights  
6      Queen's Park, Ontario Provincial Government  
..                                             ...  
153                                    Upper Rouge  
156                    St. James Town, Cabbagetown  
157         First Canadian Place, Underground city  
160  The Kingsway, Montgomery Road, Old Mill North  
165                           Church and Wellesley  

[100 rows x 3 columns]

In [15]:
temp = df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ','.join(x))
temp = temp.reset_index(drop=False)
temp.rename(columns={'Neighborhood': 'Neighborhood_joined'}, inplace=True)

temp.head()

Postalcode                     Neighborhood_joined
0        M1B                          Malvern, Rouge
1        M1C  Rouge Hill, Port Union, Highland Creek
2        M1E       Guildwood, Morningside, West Hill
3        M1G                                  Woburn
4        M1H                               Cedarbrae

In [16]:
df_merge = pd.merge(df, temp, on='Postalcode')
df_merge.drop(['Neighborhood'], axis = 1, inplace=True)
df_merge.drop_duplicates(inplace = True)
df_merge.rename(columns = {'Neighborhood_joined': 'Neighborhood'}, inplace=True)
df_merge.head()

Postalcode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [17]:
df_merge.shape

(103, 3)

In [18]:
import geocoder
import folium

In [30]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
geo_merge = pd.merge(geo_df, df_merge, on='Postalcode')
geo_merge.head(100)

toronto_df = geo_merge[geo_merge['Borough'].str.contains("Toronto")]
toronto_df.reset_index(drop=True, inplace=True)

print(toronto_df.head(100))

   Postalcode   Latitude  Longitude           Borough  \
0         M4E  43.676357 -79.293031      East Toronto   
1         M4K  43.679557 -79.352188      East Toronto   
2         M4L  43.668999 -79.315572      East Toronto   
3         M4M  43.659526 -79.340923      East Toronto   
4         M4N  43.728020 -79.388790   Central Toronto   
5         M4P  43.712751 -79.390197   Central Toronto   
6         M4R  43.715383 -79.405678   Central Toronto   
7         M4S  43.704324 -79.388790   Central Toronto   
8         M4T  43.689574 -79.383160   Central Toronto   
9         M4V  43.686412 -79.400049   Central Toronto   
10        M4W  43.679563 -79.377529  Downtown Toronto   
11        M4X  43.667967 -79.367675  Downtown Toronto   
12        M4Y  43.665860 -79.383160  Downtown Toronto   
13        M5A  43.654260 -79.360636  Downtown Toronto   
14        M5B  43.657162 -79.378937  Downtown Toronto   
15        M5C  43.651494 -79.375418  Downtown Toronto   
16        M5E  43.644771 -79.37

In [27]:
# Create Toronto map
map_toronto = folium.Map(location=[43.65, -79.38], zoom_start=10)

# Add markers
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto